<a href="https://colab.research.google.com/github/zilingggg/SAM/blob/main/SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install gradio opencv-python pillow matplotlib

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-2rob0tfx
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-2rob0tfx
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done


In [12]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

--2025-12-10 06:07:52--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.14, 3.163.189.96, 3.163.189.51, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth.1’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   108MB/s    in 20s     

2025-12-10 06:08:12 (120 MB/s) - ‘sam_vit_h_4b8939.pth.1’ saved [2564550879/2564550879]



In [16]:
import gradio as gr
import numpy as np
import cv2
from PIL import Image
from segment_anything import sam_model_registry, SamPredictor
import torch

# Load SAM
sam_checkpoint = "sam_vit_h_4b8939.pth"
device = "cuda" if torch.cuda.is_available() else "cpu"

sam = sam_model_registry["vit_h"](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)

# global state 用來存 mask
saved_mask = None

def segment_with_points(img, points):
    global saved_mask

    if img is None:
        return None

    image = np.array(img)
    predictor.set_image(image)

    if points is None or len(points) == 0:
        return img

    input_points = np.array([[p["x"], p["y"]] for p in points])
    input_labels = np.ones(len(points))

    masks, scores, _ = predictor.predict(
        point_coords=input_points,
        point_labels=input_labels,
        multimask_output=False,
    )

    mask = masks[0]
    saved_mask = mask   # ←← 存起來供下載使用

    # 顏色疊圖
    colored = (mask * 255).astype(np.uint8)
    colored = cv2.applyColorMap(colored, cv2.COLORMAP_JET)
    result = cv2.addWeighted(image, 0.6, colored, 0.4, 0)

    return Image.fromarray(result)

def save_mask():
    global saved_mask
    if saved_mask is None:
        return None

    mask_img = (saved_mask * 255).astype(np.uint8)
    pil_mask = Image.fromarray(mask_img)

    file_name = "sam_mask.png"
    pil_mask.save(file_name)
    return file_name

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### SAM 互動分割（可下載 Mask）")

    with gr.Row():
        input_image = gr.Image(type="pil", label="上傳影像")
        output_image = gr.Image(label="分割結果")

    points = gr.State([])

    # 點選事件
    def add_point(img, evt: gr.SelectData, points):
        points = points + [{"x": evt.index[0], "y": evt.index[1]}]
        return points, segment_with_points(img, points)

    input_image.select(add_point, [input_image, points], [points, output_image])

    save_btn = gr.Button("下載分割 mask")
    mask_file = gr.File()

    save_btn.click(save_mask, outputs=mask_file)

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://53fbe24fb081fd5cdf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://53fbe24fb081fd5cdf.gradio.live
